In [5]:
# 在jupyter中显示图像
%matplotlib inline  
import random
import torch
from d2l import torch as d2l

## 生成数据集

自己制作一个数据集

(**我们使用线性模型参数$\mathbf{w} = [2, -3.4]^\top$、$b = 4.2$
和噪声项$\epsilon$生成数据集及其标签：

$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$$
**)

$\epsilon$可以视为模型预测和标签时的潜在观测误差。



In [10]:
def sysnthetic_data(w,b,num_examples):
    """生成y=Xw+b+噪声"""
    # torch.normal(均值，标准差，形状)
    # 均值为0，标准差为1，服从正态分布，形状为num_examples行，len(w)列
    X=torch.normal(0,1,(num_examples,len(w)))
    # torch.matmul矩阵相乘与torch.mm矩阵相乘的区别是
    # 前者可以进行广播，后者要满足矩阵乘法唯独要求，一般用前者
    y=torch.matmul(X,w)+b
    # 生成噪音
    # 随机生成一个形状为y.shape的张量，服从正态分布，均值为0，标准差为0.01
    y+=torch.normal(0,0.01,y.shape)

    # print(y[0],y.shape)
    # print(y[0].reshape((-1,1)),y.reshape((-1,1)).shape)

    # 弄成一个元组返回，方便后面的函数调用
    # 返回一个元组，元组中有两个元素，第一个元素是X，第二个元素是y

    # y为什么要reshape((-1,1))？
    # 因为后面的函数要求y的形状为num_examples行，1列
   
    # reshape((-1,1))表示行数不确定，列数为1
    return X,y.reshape((-1,1))

# test function
# inputs,outputs=sysnthetic_data(torch.tensor([2,-3.4]),5,1000)


In [11]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = sysnthetic_data(true_w,true_b,1000)

注意，[**`features`中的每一行都包含一个二维数据样本，
`labels`中的每一行都包含一维标签值（一个标量）**]。

## 读取数据集

在下面的代码中，我们[**定义一个`data_iter`函数，
该函数接收批量大小、特征矩阵和标签向量作为输入，生成大小为`batch_size`的小批量，且做shuffle**]。
每个小批量包含一组特征和标签。

In [13]:
# num_examples = len(features)
# print(range(num_examples))
# indices = list(range(num_examples))
# print(indices)

range(0, 1000)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,

- `data_iter`有`data.DataLoader`替代

In [15]:
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    # indices用来存储所有的样本的索引
    # range(num_examples)表示生成从0到num_examples-1
    # 不过返回的是一个range对象，所以要list转为列表
    indices = list(range(num_examples))
    # 打乱样本的索引
    # 就可以实现随机读取小批量样本的效果！！！！！！！！！秒！！！！
    random.shuffle(indices)
    # 从0到num_examples，步长为batch_size
    # 每次batch_size取一次，一次取batch_size个样本
    for i in range(0,num_examples,batch_size):
        # 生成本次要取的样本的索引
        # 从indices中取出i到i+batch_size的索引
        batch_indices = torch.tensor(
            indices[i:min(i+batch_size,num_examples)])
        
        # 根据索引取出对应的样本

        # yield和return都是函数返回
        # yield关键字可以实现迭代器的效果，return也可以
        # 不过yield是一次次返回一个迭代结果，return是一次性返回
        # 其它都一样
        
        # 会让函数变成一个生成器，间隔返回多次
        # 每次迭代返回一个元组，元组中有两个元素，第一个元素是features，第二个元素是labels
        yield features[batch_indices],labels[batch_indices]

- $∇(af(x) + bg(x)) = a∇f(x) + b∇g(x)$

- 这意味着对于两个函数的线性组合，其梯度等于各个函数梯度的线性组合。
  - 所以一个batch的所有样本的loss加起来，然后再求梯度没有影响
  - 在更新参数时，要总的梯度要除以一下batch_size/或者说累加的样本个数
    - 就得到这次要更新的梯度大小了
    - 一般在优化器中实现

In [17]:
# 测试一下data_iter函数
batch_size = 10
for X,y in data_iter(batch_size,features,labels):
    print(X,'\n',y)
    break # 只打印一次


tensor([[-0.7651, -0.6104],
        [-1.9122, -0.1927],
        [-1.8469,  0.0580],
        [-1.3669, -0.0158],
        [-0.3262, -0.9734],
        [-0.2292,  0.7322],
        [-0.3754,  1.2383],
        [ 0.3689,  2.7600],
        [-0.0818, -0.2172],
        [ 0.3366, -0.1783]]) 
 tensor([[ 4.7528],
        [ 1.0344],
        [ 0.2921],
        [ 1.5171],
        [ 6.8578],
        [ 1.2424],
        [-0.7674],
        [-4.4536],
        [ 4.7858],
        [ 5.4874]])


- 上面实现的迭代其实效率很低，只是演示很好理解
- 框架中的内置迭代器效率要高很多
  - 它可以处理存储在文件中的数据和数据流提供的数据
  - 从而不用向上面那样把数据全部加载在内存中，干等着被使用

## 初始化模型参数
- 从均值为0、标准差为0.01的正态分布中采样随机数来初始化权重，
并将偏置初始化为0。

- **修改一下超参，从新跑的话，需要重新初始化模型参数**

In [84]:
w = torch.normal(0,0.01,size=(2,1),requires_grad=True)

# 测试w也初始化为0 
# w = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1,requires_grad=True)

- 我们的任务是更新这些参数，直到这些参数足够拟合我们的数据。
  - 每次更新都需要计算损失函数关于模型参数的梯度。
  - 有了这个梯度，我们就可以向减小损失的方向更新每个参数。
  - 因为手动计算梯度很枯燥而且容易出错，所以没有人会手动计算梯度。
    - 使用pytorch的autograd计算梯度

## 定义模型
- 定义模型的目标是
  - **将模型的输入和参数同模型的输出关联起来**

- `nn.Linear()`内部的实现其实也就类似思路

In [19]:
def linreg(X,w,b):
    """线性回归模型"""
    return torch.matmul(X,w) + b

## 定义损失函数
- 需要计算损失函数的梯度
  - 所以要先定义损失函数
  - 要保证真实值`y`的形状与预测值`y_hat`的形状相同
    - 就用了`reshape`函数
- 在实际中我们会一个batch所有样本的loss求和，再求梯度
  - 因为梯度的性质：函数的线性组合不会对梯度有影响，所以可以求和再求梯度
  - 但在更新参数时，要总的梯度要除一下累加的样本个数
  - 一般在优化器中实现

- `nn.MSELoss()`替代

In [22]:
def squared_loss(y_hat,y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape))**2/2

## 定义优化算法

- 下面的函数实现小批量随机梯度下降更新。
- 该函数接受模型参数集合、学习速率和批量大小作为输入。
  - 每一步更新的大小由学习速率`lr`决定。
- 用批量大小（`batch_size`）来规范化步长，
  - 因为我们计算的损失是一个批量样本的总和，所以我们用批量大小（`batch_size`）来规范化步长，这样步长大小就不会取决于我们对批量大小的选择。

- `torch.optim.SGD()`替代
- `params`有`net.parameters()`替代

In [35]:
# python定义数据类型不需要指定数据类型
# params随便是什么类型都可以，因为下面用了迭代，所以只要可以迭代就行
def sgd(params,lr,batch_size):
    """小批量随机梯度下降"""
    # 参数更新不需要记录梯度，所以使用torch.no_grad()
    with torch.no_grad():
        # 迭代params中的每一个参数
        # [w,b]就第一次迭代w，第二次迭代b
        # 这里在sgd函数内部可以修改到w,b的值
        # 是因为w,b是可变对象，相当于引用传递，所以可以修改到
        # 可变对象包括list，dict，set，bytearray
        # 不可变对象相当于按值传递，包括int，float，bool，str，tuple，frozenset，bytes
        for param in params:
            param -= lr * param.grad / batch_size
            # 梯度清零
            # 这里就放在sgd里面了
            param.grad.zero_() 


## 训练


### 超参数

- python
  - 调用函数不带括号
    - 是对函数地址的存储
    - 相当于给函数起个别名来使用
  - 带括号
    - 是对函数返回值，结果的存储
    - 调用的是函数的执行结果，需等待函数执行完毕的结果。

- 学习率太大了，就会nan
  - Not a number
    - 越界，或者除0这些
  - 比如这个设置lr=10


In [78]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size=10

如果只做了前向传播是不用梯度清零的

做了`backward()`计算了梯度，才会累加，需要清零

In [85]:
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        # w,b是初始化的权重和偏差
        # X，y是迭代器返回的本次batch的样本，标签
        # 传入网络得到预测值
        # 预测值和标签传入loss函数得到这个batch的损失
        l = loss(net(X,w,b),y)
        # l是向量，求和得到标量，再求梯度
        l.sum().backward()
        # 使用参数的梯度更新参数
        # sgd里面做了梯度清零
        sgd([w,b],lr,batch_size)
    # 每个epoch打印一次损失
    with torch.no_grad():
        # 用整个数据集计算损失
        # loss函数返回的是列表，里面每个值是一个样本的损失
        # 这里是只做了前向传播不用梯度清零
        train_l = loss(net(features,w,b),labels)
        # ":f"是用于指定浮点数的显示格式，控制浮点数在打印输出时的小数位数。
        print(f"epoch {epoch+1} ,loss {float(train_l.mean()):f}")

epoch 1 ,loss 0.052212
epoch 2 ,loss 0.000236
epoch 3 ,loss 0.000049


- 自定按函数合成的数据集，知道真正的参数是什么
  - 可以检验一下

In [59]:
print(f"w的估计误差:{true_w - w.reshape(true_w.shape)}")
print(f"b的估计误差:{true_b -b}")

w的估计误差:tensor([ 0.0004, -0.0012], grad_fn=<SubBackward0>)
b的估计误差:tensor([0.0007], grad_fn=<RsubBackward1>)


- 可看到解决还不错
  - 但是有点误差
  
## 不应该追求能完美求解参数
- 因为网络的表征范围内，有很多参数组合能完成任务
  

## 练习
1.如果我们将权重初始化为零，会发生什么。算法仍然有效吗？
- 实验来看是有效的
  
2.如果样本个数不能被批量大小整除，data_iter函数的行为会有什么变化？
- 测试了一下，没啥变化
  - 具体到函数里其实就是对for循环最后一次取不到batch个值有些影响
  - 但python的for循环应该是设计好了不会越界啥的。